## Задание 1.
### Обязательная часть
Будем парсить страницу со свежеми новостями на [habr.com/ru/all/](habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

### Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
URL = 'https://habr.com/ru/all/'
KEYWORDS = ['python', 'парсинг', 'машинное обучение', 'плафон']

In [3]:
soup = BeautifulSoup(requests.get(URL).text)
articles = soup.find_all('article')

In [4]:
def df_with_keywords(soup, KEYWORDS=' '):
    from_habr = pd.DataFrame()
    for article in soup:
        if article.find('h2') and article.find('h2').find('a'):
            link = article.find('h2').find('a')['href']
        else:
            link = 'not found'
        if article.find('h2'):
            header = article.find('h2').text.strip()
        else:
            header = 'not found'
        if article.find('ul', class_='post__hubs'):
            hubs = article.find('ul', class_='post__hubs').text.replace('\n', '').strip()
        else:
            hubs = 'not found'
        if link:
            soup = BeautifulSoup(requests.get(link).text)
            if soup.find('span', class_='post__time'):
                date = soup.find('span', class_='post__time')['data-time_published']
                date_only = date[:date.index('T')]
            else:
                date_only = 'not found'
            if soup.find('div', class_ = 'post__text'):
                text = soup.find('div', class_ = 'post__text').text.strip()
            else:
                text = 'not_found'
            row = {'date': date_only, 'header': header, 'link': link, 'text': text}
        else:
            row = {'date': 'not found', 'header': header, 'link': link, 'text': 'not found'}
        for word in KEYWORDS:
            if word in header.lower() or word in hubs.lower() or word in text.lower():
                from_habr = pd.concat([from_habr, pd.DataFrame([row])], ignore_index=True)
    return from_habr

In [5]:
df_with_keywords(articles, KEYWORDS)

,date,header,link,text
0,2021-02-18,Как классифицировать мусор с помощью Raspberry...,https://habr.com/ru/company/skillfactory/blog/...,ML на основе нейросетей открывает для программ...
1,2021-02-18,Компьютерное зрение в промышленной дефектоскоп...,https://habr.com/ru/company/itmai/blog/543000/,"В предыдущей заметке мы рассказали о том, как ..."
2,2021-02-18,Компьютерное зрение в промышленной дефектоскоп...,https://habr.com/ru/company/itmai/blog/543000/,"В предыдущей заметке мы рассказали о том, как ..."
3,2021-02-18,Как распознать рукописный текст с помощью ИИ н...,https://habr.com/ru/company/skillfactory/blog/...,Распознавание рукописных цифр с помощью Tensor...
4,2021-02-18,"LuxCity — стратегия для разработчиков, где код...",https://habr.com/ru/company/luxoft/blog/543010/,"Процесс знакомства с компаниями, прохождение м..."


In [6]:
df_with_keywords(articles)

,date,header,link,text
0,2021-02-18,[Личный опыт] Свой среди чужих: как встроиться...,https://habr.com/ru/company/gms/blog/543176/,В блоге мы много говорим про релокейт. Опыт у ...
1,2021-02-18,"Как выжить, если вы падаете со скоростью 190 к...",https://habr.com/ru/post/543174/,"Вы оказались на высоте десяти километров, и па..."
2,2021-02-18,Вариант записи текстов на русском языке с помо...,https://habr.com/ru/post/543172/,Статья написана в порядке «зарядки для хвоста ...
3,2021-02-18,Как мы боролись с фейковыми аккаунтами на сайт...,https://habr.com/ru/company/itsoft/blog/543166/,С ростом популярности сервиса мы столкнулись с...
4,2021-02-18,Как устроены одноразовые электронные сигареты,https://habr.com/ru/post/543170/,В прошлом году одноразовые электронные сигарет...
5,2021-02-18,Объяснение легковесных потоков в 200 строк на ...,https://habr.com/ru/post/543158/,Объяснение легковесных потоков в 200 строк на ...
6,2021-02-18,Как классифицировать мусор с помощью Raspberry...,https://habr.com/ru/company/skillfactory/blog/...,ML на основе нейросетей открывает для программ...
7,2021-02-18,Злоумышленники начали активно разрабатывать на...,https://habr.com/ru/company/selectel/blog/543152/,Один из доводов «маководов» в пользу своей сис...
8,2021-02-18,Коммерческая тайна. Частые вопросы и ответы,https://habr.com/ru/post/538572/,Коммерческая тайна вызывает множество вопросов...
9,2021-02-18,Грубая оценка проблемности GitHub-проектов,https://habr.com/ru/post/543142/,"Здравствуйте, меня зовут Дмитрий Карловский и ..."


## Задание 2.
### Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Check. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [7]:
from bs4 import BeautifulSoup
import json
import pandas as pd
import requests

In [8]:
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
emails = {'emailAddresses': ['xxx@x.ru', 'yyy@y.com']}
headers = {'Vaar-Version': '0', 'Vaar-Header-App-Product': 'hackcheck-web-avast'}

In [9]:
breaches = requests.post(URL, json=emails, headers=headers)
breaches.text

'{"breaches":{"16613":{"breachId":16613,"site":"verifications.io","recordsCount":677914246,"description":"Big data e-mail verification platform verifications.io leaked a database containing sensitive PII belonging to over 600 million victims. The breach was discovered by a security researcher who found an unsecured 150GB MongoDB database, which he was able to track to an email verification service called Verifications.io. The leaked database contained information such as physical addresses, phone numbers, email addresses, dates of birth, gender, employer and job information, geographic location and IP addresses.\\r\\n\\r\\n","publishDate":"2019-03-28T00:00:00Z","statistics":{"usernames":0,"passwords":0,"emails":677914246}},"17609":{"breachId":17609,"site":"vk.com","recordsCount":42536910,"description":"At some time in 2020, the Russian social networking website vKontakte was allegedly breached. The stolen data contains email addresses and additional personal information. This breach is

In [10]:
breach = json.loads(breaches.text)
breach['summary']

{'xxx@x.ru': {'breaches': [16613, 17609, 3176, 12, 3, 3164, 2961, 15]},
 'yyy@y.com': {'breaches': [16613,
   16488,
   18155,
   17110,
   17670,
   13094,
   13254,
   37177,
   16768,
   41,
   2,
   3587,
   17009,
   16802,
   3,
   13662,
   3520,
   15,
   3669]}}

In [11]:
for key in breach['breaches']:
    print(key)
    print(breach['breaches'][key])

16613
{'breachId': 16613, 'site': 'verifications.io', 'recordsCount': 677914246, 'description': 'Big data e-mail verification platform verifications.io leaked a database containing sensitive PII belonging to over 600 million victims. The breach was discovered by a security researcher who found an unsecured 150GB MongoDB database, which he was able to track to an email verification service called Verifications.io. The leaked database contained information such as physical addresses, phone numbers, email addresses, dates of birth, gender, employer and job information, geographic location and IP addresses.\r\n\r\n', 'publishDate': '2019-03-28T00:00:00Z', 'statistics': {'usernames': 0, 'passwords': 0, 'emails': 677914246}}
17609
{'breachId': 17609, 'site': 'vk.com', 'recordsCount': 42536910, 'description': 'At some time in 2020, the Russian social networking website vKontakte was allegedly breached. The stolen data contains email addresses and additional personal information. This breach i

In [12]:
for key in breach['breaches']:
    for item in breach['breaches'][key]:
        print(item)

breachId
site
recordsCount
description
publishDate
statistics
breachId
site
recordsCount
description
publishDate
statistics
breachId
site
recordsCount
description
publishDate
statistics
breachId
site
recordsCount
description
publishDate
statistics
breachId
site
recordsCount
description
publishDate
statistics
breachId
site
recordsCount
description
publishDate
statistics
breachId
site
recordsCount
description
publishDate
statistics
breachId
site
recordsCount
description
publishDate
statistics
breachId
site
recordsCount
description
publishDate
statistics
breachId
site
recordsCount
description
publishDate
statistics
breachId
site
recordsCount
description
publishDate
statistics
breachId
site
recordsCount
description
publishDate
statistics
breachId
site
recordsCount
description
publishDate
statistics
breachId
site
recordsCount
description
publishDate
statistics
breachId
site
recordsCount
description
publishDate
statistics
breachId
site
recordsCount
description
publishDate
statistics
breachId

In [13]:
from_avast = pd.DataFrame()
for key in breach['summary']:
    email = key
    for id_ in breach['summary'][email]['breaches']:
        date = breach['breaches'][str(id_)]['publishDate']
        source = breach['breaches'][str(id_)]['site']
        description = breach['breaches'][str(id_)]['description']
        row = {'email': email, 'date': date, 'source': source, 'description': description}
        from_avast = pd.concat([from_avast, pd.DataFrame([row])], ignore_index=True)
from_avast

,email,date,source,description
0,xxx@x.ru,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
1,xxx@x.ru,2020-05-21T00:00:00Z,vk.com,"At some time in 2020, the Russian social netwo..."
2,xxx@x.ru,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
3,xxx@x.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
4,xxx@x.ru,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
5,xxx@x.ru,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
6,xxx@x.ru,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
7,xxx@x.ru,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
8,yyy@y.com,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
9,yyy@y.com,2019-02-21T00:00:00Z,www.dangdang.com,"This is a list of email addresses only, and as..."


### Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [1]:
from datetime import datetime
import json
import pandas as pd
import requests

In [3]:
from web_scrapping_params import TOKEN
URL_REQUEST = 'https://api.vk.com/method/wall.get?'
GROUP = '-30159897'
VERSION = '5.52'

In [4]:
params = {
    'owner_id': GROUP,
    'count': 50,
    'access_token': TOKEN,
    'v': VERSION
}

In [5]:
res = requests.get(URL_REQUEST, params)
res.json()

{'response': {'count': 7503,
  'items': [{'id': 78384,
    'from_id': -30159897,
    'owner_id': -30159897,
    'date': 1613572380,
    'marked_as_ads': 0,
    'post_type': 'post',
    'text': 'Представьте, что вы трудитесь в секретной организации и никто не должен знать, где вы работаете 🤫 \n \nРасскажите о своём рабочем месте так, чтобы всем захотелось там оказаться. \nА мы попробуем угадать, кем вы работаете 😉\n \nПоехали 👇',
    'is_pinned': 1,
    'attachments': [{'type': 'photo',
      'photo': {'album_id': -7,
       'date': 1613552271,
       'id': 457247497,
       'owner_id': -30159897,
       'has_tags': False,
       'access_key': '3c4b45239ac2d478aa',
       'height': 1080,
       'photo_1280': 'https://sun9-24.userapi.com/impg/hkW_3WTxLRplOAhynDlfp6WwCnj7JO3ZWF0ScA/1DAW7wy71vw.jpg?size=1080x1080&quality=96&proxy=1&sign=b85a7be3134cf96be19168944168e83d&c_uniq_tag=QwxzledQg6RwsxKfDG-HXN03d8eKsy-2GKjTmiWP5-M&type=album',
       'photo_130': 'https://sun9-24.userapi.com/impg/

In [30]:
posts = pd.DataFrame(res.json()['response']['items'])[['date', 'text']]

In [32]:
posts['date'] = posts['date'].apply(lambda t: datetime.strftime(datetime.fromtimestamp(t), '%Y-%m-%d'))

In [33]:
posts

,date,text
0,2021-02-17,"Представьте, что вы трудитесь в секретной орга..."
1,2021-02-18,"Пятница, 16:30, а в списке задач ещё четыре ст..."
2,2021-02-18,"В свежем Data Science-дайджесте рассказали, в ..."
3,2021-02-17,📆 22 февраля стартует бесплатный курс «Как ста...
4,2021-02-16,Однажды в вашей карьере настанет момент делеги...
5,2021-02-16,На прошлой неделе в мире диджитала произошло м...
6,2021-02-15,"Теперь можно не только слушать подкасты, но и ..."
7,2021-02-15,Направлений в дизайне много: можно делать рекл...
8,2021-02-14,«Жизнь как коробка шоколадных конфет: никогда ...
9,2021-02-13,*партнёрский пост* \n \n26-28 февраля пройдёт ...
